In [1]:
import os
from urllib.request import urlretrieve
import numpy as np
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain_community.llms import HuggingFacePipeline
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.document_loaders import PyPDFDirectoryLoader
from langchain_community.document_loaders import UnstructuredExcelLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

In [ ]:
# from langchain_community.llms import HuggingFaceHub

# '''
# 请注意，这里对于一般的机器，内存要求极高，我再colab尝试下载中文LLM的baichuan，但是仍然OOM，所以用的一个在线LLM做测试，如果在一些网络限制比较
# 严格的地方，需要开发者自己从开源社区提前下载好对应的LLM
# '''
# huggingface_api_token = os.getenv('HUGGINGFACE_API_TOKEN')

# hf = HuggingFaceHub(
#     huggingfacehub_api_token=huggingface_api_token,
#     repo_id="mistralai/Mistral-7B-v0.1",
#     model_kwargs={"temperature":0.1, "max_length":2000})

# query = """电力市场与普通商品市场的差异"""  # Sample question, change to other questions you are interested in.
# hf.invoke(query)

In [ ]:
# from zhipuai_llm import ZhipuAILLM
# zhipuai_api_key = os.getenv("ZHIPUAI_API_KEY")

# zhipuai_model = ZhipuAILLM(model='glm-4', temperature=0.1, api_key=zhipuai_api_key)
# res = zhipuai_model("电力市场与普通商品市场的差异")
# print(res)

In [ ]:
from langchain_community.llms import BaichuanLLM

# 加载百川的LLM
baichaun = BaichuanLLM()

res = baichaun.invoke("电力市场与普通商品市场的差异")
print(res)

In [4]:
data = "../demo/"
# 这里为了做演示，随便找了一个in house 文本/content/0858能源动力-博士专业学位创新成果具体标准2024(最终).pdf

In [5]:
loader = PyPDFDirectoryLoader(data)

docs_before_split = loader.load()
# 过滤目录和附录
docs_before_split = [doc for doc in docs_before_split if doc.metadata['page'] > 7 and doc.metadata['page'] <275] 
# # 这里可以理解为对自己的文档切分成块，chuck size是每一块的大小，可以根据需求调整

text_splitter = CharacterTextSplitter(chunk_size = 256,
                                    chunk_overlap  = 30,)

# docs_after_split = text_splitter.split_documents(docs_before_split)

# docs_after_split[0]

Document(page_content='第一章 电力现货市场基础   \n             \n1   \n第一章 \n \n \n电力现货市场基础  \n \n \n \n  \n 1. 什么是电力市场？电力市场与 普通商品市场有哪 些差异？电力市场有\n哪些特征？ \n（1）电力市场的概念。  \n我国关于电力市场的权威解释始见于《中国电力百科全书  电力系统卷（第二版） 》。\n电力市场的定义为：基于市场经济原则，电力市场的定义为基于市场经济原则，为实现', metadata={'source': '..\\demo\\电力现货市场101问.pdf', 'page': 8})

In [ ]:
# 提取所有文本信息
full_text=""
for doc in docs_before_split:
    full_text += doc.page_content
full_text

In [6]:
len(docs_after_split)

1604

In [60]:

# loader = UnstructuredExcelLoader(data, mode="elements")
# docs = loader.load()
# docs[0]
# docs_before_split = docs[0].page_content
# #对数据进行分割
# docs_after_split = docs_before_split.strip("\n\n\n")
# docs_after_split = docs_after_split.split("\n\n\n")
# print(len(docs_after_split))

In [62]:
# from langchain_core.documents import Document
# # 重新封装成Document类
# docs_after_split = [ Document(page_content=str, metadata={"source": idx}) for idx,str in enumerate(docs_after_split)]
# docs_after_split[0]

Document(page_content='Question\nAnswer', metadata={'source': 0})

In [7]:
# 从过往工作经验看，embedding对于rag效果影响比较大，一般首选还是openi embedding做这一部分，开源的效果很一般
huggingface_embeddings = HuggingFaceBgeEmbeddings(
    model_name="moka-ai/m3e-base",  # 选用m3e模型做embedding
    model_kwargs={'device':'cpu'},
    encode_kwargs={'normalize_embeddings': True}
)

d:\miniconda3\envs\transformers\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
d:\miniconda3\envs\transformers\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
sample_embedding = np.array(huggingface_embeddings.embed_query(docs_after_split[0].page_content))
print("Sample embedding of a document chunk: ", sample_embedding)
print("Size of the embedding: ", sample_embedding.shape)

In [9]:
# 这一步是对自己前一步切好的文本做embedding处理，并通过faiss存储，faiss的主要作用是提高检索效率
vectorstore = FAISS.from_documents(docs_after_split, huggingface_embeddings)

In [15]:
query = """电力市场与普通商品市场的差异"""
         # Sample question, change to other questions you are interested in.
relevant_documents = vectorstore.similarity_search(query)
for i, doc in enumerate(relevant_documents):
    print(f"检索到的第{i+1}个内容: \n {doc.page_content}", end="\n-----------------------------------------------------\n")

# 从这一步看，已经很清晰rag的原理了，即从大量文件索引找到top k相关的text块，供下一步LLM查找、总结答案

检索到的第1个内容: 
 品种、交易时间、竞争模式等维度进一步细分，各分类市场的知识将在后续问题讨论中详述。  
（2）电力市场和普通商品市场的差异。  
电力市场相较于普通商品市场具有显著的特殊性， 归根到底就在于电力商品与其他一
般商品的差异性。电力商品本质即电能，与其他商品最本质的差异在于其自然属性和社
会属性。  
电力商品具有无仓储性。电能的生产、交割和消费几乎是同时完成的，其交割速度远
快于一般商品，因此也不存在一般商品一手交钱一手交货的交易方式。
-----------------------------------------------------
检索到的第2个内容: 
 电力现货市场 101 问  
  42 
 
图1 − 4 现货市场时间轴  
（1）集中式市场模式。  
市场供需双方在每天特定时间之前向调度机构报价， 由调度机构根据供需双方报价和
网络条件等出清。在不同国家，日前市场的名称有所不同。例如，在挪威和美国 PJM电
力市场称日前（ day-ahead ）交易，在澳大利亚称短期提前（ short-run ahead ）交易，在我
-----------------------------------------------------
检索到的第3个内容: 
 电力商品具有同质性。电能不带有任何生产者的标识，电能生产者将生产的电能输入
-----------------------------------------------------
检索到的第4个内容: 
 电力现货市场 101 问  
  30 和一个双边市场，给予了市场充分的流动性，也为市场成员提供了多次报价的机会。  
4）美国纽约州容量市场。  
美国纽约州 NYISO运营着一个装机容量（ install capacity ，ICAP）市场。该市场的关
键特征是它是一个短期市场，虽然装机容量市场的目的是确保长期的资源充裕性，但不
提供长期的价格信号，也不锁定未来几年的特定容量价格。  
NYISO的ICAP市场有两个为期 6个月的容量期， 即冬季和夏季容量期。 通过三次拍
-----------------------------------------------------


In [16]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})
# 这里的K与检索文本的数量相关，但一般不会太大

In [17]:
prompt_template = """
请用下面相关文本回答问题，如果不知道答案，就回复不知道，

{context}

Question: {question}

Helpful Answer:
"""

PROMPT = PromptTemplate(
 template=prompt_template, input_variables=["context", "question"]
)

In [18]:
retrievalQA = RetrievalQA.from_chain_type(
    llm=baichaun,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)

In [19]:
result = retrievalQA.invoke({"query": query})
print(result['result'])

电力市场与普通商品市场的差异主要体现在电力商品的特性上。电力商品具有无仓储性，这意味着电能的生产、交割和消费几乎是同时完成的，其交割速度远快于一般商品，因此也不存在一般商品一手交钱一手交货的交易方式。此外，电力商品具有同质性，电能不带有任何生产者的标识，电能生产者将生产的电能输入电网后，无法区分不同生产者的电能。


In [20]:
relevant_docs = result['source_documents']
print(f'There are {len(relevant_docs)} documents retrieved which are relevant to the query.')
print("*" * 100)
for i, doc in enumerate(relevant_docs):
    # print(f"Relevant Document #{i+1}:\nSource file: {doc.metadata['source']}, Page: {doc.metadata['page']}\nContent: {doc.page_content}")
    print(f"Relevant Document #{i+1}:\nSource file: {doc.metadata['source']}\nContent: {doc.page_content}")
    print("-"*100)
    print(f'There are {len(relevant_docs)} documents retrieved which are relevant to the query.')

There are 3 documents retrieved which are relevant to the query.
****************************************************************************************************
Relevant Document #1:
Source file: ..\demo\电力现货市场101问.pdf
Content: 品种、交易时间、竞争模式等维度进一步细分，各分类市场的知识将在后续问题讨论中详述。  
（2）电力市场和普通商品市场的差异。  
电力市场相较于普通商品市场具有显著的特殊性， 归根到底就在于电力商品与其他一
般商品的差异性。电力商品本质即电能，与其他商品最本质的差异在于其自然属性和社
会属性。  
电力商品具有无仓储性。电能的生产、交割和消费几乎是同时完成的，其交割速度远
快于一般商品，因此也不存在一般商品一手交钱一手交货的交易方式。
----------------------------------------------------------------------------------------------------
There are 3 documents retrieved which are relevant to the query.
Relevant Document #2:
Source file: ..\demo\电力现货市场101问.pdf
Content: 电力现货市场 101 问  
  42 
 
图1 − 4 现货市场时间轴  
（1）集中式市场模式。  
市场供需双方在每天特定时间之前向调度机构报价， 由调度机构根据供需双方报价和
网络条件等出清。在不同国家，日前市场的名称有所不同。例如，在挪威和美国 PJM电
力市场称日前（ day-ahead ）交易，在澳大利亚称短期提前（ short-run ahead ）交易，在我
----------------------------------------------------------------------------------------------------
There are 3 documents 

In [ ]:
'''
fine tuning和rag还是要经过实验对比，具体看效果，在LLM之前，大家比较习惯都先fine tuning下（比如BERT系列），但从上面仅仅是下载模型，
colab都OOM了看，fine tuning对于机器的要求更高，而且数据较少的情况下效果不好说。

也可以fine tuning之后，再结合fine tuning之后的LLM做rag

'''